# 📥 RAG: Data Ingestion (PDF & Web)
Notebook นี้จะสาธิตวิธีการดึงข้อมูลจาก **PDF** และ **Website** เพื่อนำมาทำ RAG ครับ

## 1. การดึงข้อมูลจาก PDF (PDF Loader)

In [4]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# แบบที่ 1: อ่านทีละไฟล์
# loader = PyPDFLoader("path/to/your/file.pdf")
# docs = loader.load()

# แบบที่ 2: อ่านทั้งโฟลเดอร์ (แนะนำสำหรับ Project นี้)
# เอาไฟล์ PDF ไปวางใน folder 'data/recipes' ก่อนนะ
try:
    loader = PyPDFDirectoryLoader("data/recipes")
    pdf_docs = loader.load()
    print(f"✅ PDF Loaded: {len(pdf_docs)} pages")
    if pdf_docs:
        print("Sample content:", pdf_docs[0].page_content[:200])
except Exception as e:
    print(f"❌ Error loading PDF: {e}")

✅ PDF Loaded: 0 pages


## 2. การดึงข้อมูลจาก Website (Web Scraper)
ใช้ `WebBaseLoader` ของ LangChain ซึ่งเก่งมากในการดึง Text ออกจาก HTML

In [6]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# ใส่ URL ที่ต้องการดึง (เช่น เว็บราคากลาง หรือ บทความ)
url = "https://kohplanner.com/food/10-classic-thai-dishes-recipes/" # ตัวอย่างเว็บตลาดไท

# ใช้ bs4.SoupStrainer เพื่อกรองเอาเฉพาะส่วนเนื้อหา (ตัดพวก Menu, Footer ออก)
# ถ้าไม่กรอง มันจะติด Code ขยะมาเยอะ
bs4_strainer = bs4.SoupStrainer(class_=("productName")) 

loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs={"parse_only": bs4_strainer}, # เปิดบรรทัดนี้ถ้าต้องการกรอง CSS Class เฉพาะเจาะจง
)

web_docs = loader.load()

print(f"✅ Web Loaded: {len(web_docs)} documents")
print("Sample content:", web_docs[0].page_content[:500].strip())

✅ Web Loaded: 1 documents
Sample content: 


## 3. รวมร่าง (Merge) และเตรียม Vector Store
เอาข้อมูลจากทั้ง PDF และ Web มารวมกันเพื่อส่งให้ AI

In [7]:
# รวมข้อมูล
all_docs = pdf_docs + web_docs
print(f"Total Documents: {len(all_docs)}")

# ขั้นตอนต่อไปคือ Split Text -> Embedding -> Vector Store (เหมือนเดิม)

Total Documents: 1
